In [33]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 25: Clock Signal ---</h2><p>You open the door and find yourself on the roof. The city sprawls away from you for miles and miles.</p>
<p>There's not much time now - it's already Christmas, but you're nowhere near the North Pole, much too far to deliver these stars to the sleigh in time.</p>
<p>However, maybe the <em>huge antenna</em> up here can offer a solution. After all, the sleigh doesn't need the stars, exactly; it needs the timing data they provide, and you happen to have a massive signal generator right here.</p>
<p>You connect the stars you have to your prototype computer, connect that to the antenna, and begin the transmission.</p>
<p><span title="Then again, if something ever works on the first try, you should be *very* suspicious.">Nothing happens.</span></p>
<p>You call the service number printed on the side of the antenna and quickly explain the situation. "I'm not sure what kind of equipment you have connected over there," he says, "but you need a clock signal." You try to explain that this is a signal for a clock.</p>
<p>"No, no, a <a href="https://en.wikipedia.org/wiki/Clock_signal">clock signal</a> - timing information so the antenna computer knows how to read the data you're sending it. An endless, alternating pattern of <code>0</code>, <code>1</code>, <code>0</code>, <code>1</code>, <code>0</code>, <code>1</code>, <code>0</code>, <code>1</code>, <code>0</code>, <code>1</code>...." He trails off.</p>
<p>You ask if the antenna can handle a clock signal at the frequency you would need to use for the data from the stars. "There's <em>no way</em> it can! The only antenna we've installed capable of <em>that</em> is on top of a top-secret Easter Bunny installation, and you're <em>definitely</em> not-" You hang up the phone.</p>
<p>You've extracted the antenna's clock signal generation <a href="12">assembunny</a> code (your puzzle input); it looks mostly compatible with code you worked on <a href="23">just recently</a>.</p>
<p>This antenna code, being a signal generator, uses one extra instruction:</p>
<ul>
<li><code>out x</code> <em>transmits</em> <code>x</code> (either an integer or the <em>value</em> of a register) as the next value for the clock signal.</li>
</ul>
<p>The code takes a value (via register <code>a</code>) that describes the signal to generate, but you're not sure how it's used. You'll have to find the input to produce the right signal through experimentation.</p>
<p><em>What is the lowest positive integer</em> that can be used to initialize register <code>a</code> and cause the code to output a clock signal of <code>0</code>, <code>1</code>, <code>0</code>, <code>1</code>... repeating forever?</p>
</article>


In [34]:
from tabulate import tabulate


def process(
    program: str, register_names: str = "abcd", register_initializer=lambda x: 0
) -> dict[str, int]:
    instructions = [i.split() for i in program.strip().splitlines()]
    registers = {l: register_initializer(l) for l in register_names}
    counter = 0
    signal_prev = None
    s = ""

    while counter < len(instructions):
        instruction = instructions[counter]
        if instruction[0] == "nop":
            counter += 1
        elif instruction[0] == "add":
            _, x, y = instruction
            if y in register_names:
                if x in register_names:
                    registers[y] += registers[x]
                else:
                    registers[y] += int(x)
            counter += 1
        elif instruction[0] == "mul":
            _, x, y = instruction
            if y in register_names:
                if x in register_names:
                    registers[y] *= registers[x]
                else:
                    registers[y] *= int(x)
            counter += 1
        elif instruction[0] == "cpy":
            # cpy x y copies x (either an integer or the value of a register)
            # into register y.
            _, x, y = instruction
            if y in register_names:
                if x in register_names:
                    registers[y] = registers[x]
                else:
                    registers[y] = int(x)
            counter += 1
        elif instruction[0] == "inc":
            # inc x increases the value of register x by one.
            _, x = instruction
            registers[x] += 1
            counter += 1
        elif instruction[0] == "dec":
            # dec x decreases the value of register x by one.
            _, x = instruction
            registers[x] -= 1
            counter += 1
        elif instruction[0] == "jnz":
            # jnz x y jumps to an instruction y away
            # (positive means forward; negative means backward),
            # but only if x is not zero.
            _, x, y = instruction
            y = registers[y] if y in register_names else int(y)
            if x in register_names and registers[x] != 0:
                counter += y
            elif x.isdecimal() and int(x) != 0:
                counter += y
            else:
                counter += 1
        elif instruction[0] == "tgl":
            # tgl x toggles the instruction x away (pointing at instructions
            # - like jnz does: positive means forward; negative means backward):
            _, x = instruction
            if x in register_names:
                v = registers[x]
            else:
                v = int(x)

            if 0 <= counter + v < len(instructions):
                # - If an attempt is made to toggle an instruction outside the program,
                #   nothing happens.

                # - The arguments of a toggled instruction are not affected.
                if len(instructions[counter + v]) == 2:
                    # - For one-argument instructions, inc becomes dec, and all other
                    #   one-argument instructions become inc.
                    # - If tgl toggles itself (for example, if a is 0, tgl a would
                    #   target itself and become inc a), the resulting instruction
                    #   is not executed until the next time it is reached.
                    if instructions[counter + v][0] == "inc":
                        instructions[counter + v][0] = "dec"
                    else:
                        instructions[counter + v][0] = "inc"
                else:
                    # - For two-argument instructions, jnz becomes cpy, and all other
                    #   two-instructions become jnz.
                    if instructions[counter + v][0] == "jnz":
                        instructions[counter + v][0] = "cpy"
                    else:
                        instructions[counter + v][0] = "jnz"
            counter += 1
        elif instruction[0] == "out":
            _, x = instruction
            if x in register_names:
                signal = registers[x]
            else:
                signal = int(x)

            if not (0 <= signal <= 1 and signal != signal_prev):
                return None
            else:
                s += f"{signal}"
                if len(s) == 100:
                    return s

            signal_prev = signal
            counter += 1
        else:
            raise ValueError(f"Unkonw isntruction: {instruction}")

    return registers


original = """
cpy a d
cpy 9 c
cpy 282 b
inc d
dec b
jnz b -2
dec c
jnz c -5
cpy d a
jnz 0 0
cpy a b
cpy 0 a
cpy 2 c
jnz b 2
jnz 1 6
dec b
dec c
jnz c -4
inc a
jnz 1 -7
cpy 2 b
jnz c 2
jnz 1 4
dec b
dec c
jnz 1 -4
jnz 0 0
out b
jnz a -19
jnz 1 -21
"""
for i in range(1000):
    s = process(original, register_initializer=lambda l: i if l == "a" else 0)
    if s is None:
        continue
    print(f"{i=}")
    print(s)
    break

i=192
0101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>192</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The antenna is ready. Now, all you need is the <em class="star">fifty stars</em> required to generate the signal for the sleigh, but you don't have enough.</p>
<p>You look toward the sky in desperation... suddenly noticing that a lone star has been installed at the top of the antenna!  Only <em>49 more</em> to go.</p>
</article>
<form method="post" action="25/answer"><input type="hidden" name="level" value="2"><input type="hidden" name="answer" value="0"><p>You have enough stars to <input type="submit" value="[Transmit the Signal]">.</p></form>
<p>Although it hasn't changed, you can still <a href="25/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+Part+One+of+%22Clock+Signal%22+%2D+Day+25+%2D+Advent+of+Code+2016&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F25&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+Part+One+of+%22Clock+Signal%22+%2D+Day+25+%2D+Advent+of+Code+2016+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F25';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>


<link href="style.css" rel="stylesheet"></link>

<main>
<article>
<p>You activate all <span class="day-success">fifty stars</span> and transmit the signal. The star atop the antenna begins to glow.</p><p><em>Suddenly, you see the sleigh fly past you!</em></p><p>Looks like Santa was already waiting for your signal.</p>
<p class="aside">Congratulations!  You've finished every puzzle in Advent of Code 2016!  I hope you had as much fun solving them as I had making them for you.  I'd love to hear about your adventure; you can get in touch with me via contact info on <a href="http://was.tl/" target="_blank">my website</a> or through <a href="https://twitter.com/ericwastl" target="_blank">Twitter</a> or <a href="https://hachyderm.io/@ericwastl" target="_blank">Mastodon</a>.</p>
<p class="aside">If you'd like to see more things like this in the future, please consider <a href="/2016/support" target="_blank">supporting</a> Advent of Code and sharing it with others.</p>
<p class="aside">To hear about future projects, you can follow me on <a href="https://twitter.com/ericwastl" target="_blank">Twitter</a> or <a href="https://hachyderm.io/@ericwastl" target="_blank">Mastodon</a>.</p>
<p class="aside">I've <span style="border-bottom:1px dotted #ffff66;" title="Yep, just like that.  There's at least one in the description for each day.">highlighted</span> the easter eggs in each puzzle, just in case you missed any.  Hover your mouse over them, and the easter egg will appear.</p>
<p>You can <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I+just+completed+all+25+days+of+Advent+of+Code+2016%21&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I+just+completed+all+25+days+of+Advent+of+Code+2016%21+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this moment with your friends, or <a href="/2016">[Go Check on Your Calendar]</a>.
</p></article>
<style>
.calendar-bkg {
  background: linear-gradient(to bottom, rgba(255,255,255,0) 0%,rgba(255,255,255,.3) 100%);
}
.sf {
  position: fixed;
  animation: anim-sf0 linear 20s infinite;
  z-index: -1;
}
.sf:before {
  content: "*";
}
.sf.sf0 { color: #ffffff; }
.sf.sf1 { color: #cccccc; animation-name: anim-sf1; }
.sf.sf2 { color: #999999; animation-name: anim-sf2; }
.sf.sf3 { color: #666666; animation-name: anim-sf3; }
@keyframes anim-sf0 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(8em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(8em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf1 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(6em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(6em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf2 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(4em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(4em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf3 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(2em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(2em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
</style>
<div class="sf sf0" style="top:-1.16em; left:21.67vw; animation-delay:-11.41s;"></div><div class="sf sf1" style="top:-2.98em; left:31.35vw; animation-delay:-19.75s;"></div><div class="sf sf1" style="top:-2.22em; left:70.27vw; animation-delay:-19.61s;"></div><div class="sf sf1" style="top:-2.99em; left:63.63vw; animation-delay:-16.66s;"></div><div class="sf sf0" style="top:-1.48em; left:88.93vw; animation-delay:-19.18s;"></div><div class="sf sf1" style="top:-2.04em; left:53.08vw; animation-delay:-17.74s;"></div><div class="sf sf3" style="top:-4.85em; left:5.05vw; animation-delay:-15.30s;"></div><div class="sf sf2" style="top:-3.30em; left:78.21vw; animation-delay:-6.71s;"></div><div class="sf sf0" style="top:-1.29em; left:50.50vw; animation-delay:-6.85s;"></div><div class="sf sf3" style="top:-4.67em; left:7.90vw; animation-delay:-9.22s;"></div><div class="sf sf1" style="top:-2.56em; left:13.63vw; animation-delay:-16.73s;"></div><div class="sf sf0" style="top:-1.95em; left:57.00vw; animation-delay:-5.97s;"></div><div class="sf sf0" style="top:-1.74em; left:44.63vw; animation-delay:-0.36s;"></div><div class="sf sf1" style="top:-2.35em; left:73.87vw; animation-delay:-9.50s;"></div><div class="sf sf3" style="top:-4.19em; left:41.63vw; animation-delay:-17.67s;"></div><div class="sf sf1" style="top:-2.47em; left:15.57vw; animation-delay:-16.74s;"></div><div class="sf sf0" style="top:-1.45em; left:54.77vw; animation-delay:-15.20s;"></div><div class="sf sf1" style="top:-2.82em; left:67.21vw; animation-delay:-8.81s;"></div><div class="sf sf3" style="top:-4.10em; left:70.85vw; animation-delay:-7.18s;"></div><div class="sf sf1" style="top:-2.24em; left:55.19vw; animation-delay:-19.31s;"></div><div class="sf sf2" style="top:-3.72em; left:36.03vw; animation-delay:-6.88s;"></div><div class="sf sf2" style="top:-3.87em; left:9.48vw; animation-delay:-12.86s;"></div><div class="sf sf2" style="top:-3.30em; left:63.83vw; animation-delay:-3.22s;"></div><div class="sf sf2" style="top:-3.72em; left:13.12vw; animation-delay:-6.71s;"></div><div class="sf sf2" style="top:-3.31em; left:70.28vw; animation-delay:-0.96s;"></div><div class="sf sf2" style="top:-3.32em; left:92.01vw; animation-delay:-10.17s;"></div><div class="sf sf1" style="top:-2.16em; left:47.84vw; animation-delay:-7.99s;"></div><div class="sf sf2" style="top:-3.82em; left:60.93vw; animation-delay:-8.66s;"></div><div class="sf sf3" style="top:-4.19em; left:88.48vw; animation-delay:-17.93s;"></div><div class="sf sf2" style="top:-3.43em; left:81.31vw; animation-delay:-2.47s;"></div><div class="sf sf2" style="top:-3.70em; left:31.24vw; animation-delay:-2.29s;"></div><div class="sf sf3" style="top:-4.87em; left:73.71vw; animation-delay:-3.72s;"></div><div class="sf sf0" style="top:-1.25em; left:52.81vw; animation-delay:-12.85s;"></div><div class="sf sf1" style="top:-2.09em; left:37.41vw; animation-delay:-19.43s;"></div><div class="sf sf2" style="top:-3.30em; left:39.19vw; animation-delay:-1.77s;"></div><div class="sf sf0" style="top:-1.69em; left:81.53vw; animation-delay:-3.27s;"></div><div class="sf sf2" style="top:-3.03em; left:89.80vw; animation-delay:-14.99s;"></div><div class="sf sf1" style="top:-2.46em; left:61.15vw; animation-delay:-4.20s;"></div><div class="sf sf2" style="top:-3.99em; left:9.15vw; animation-delay:-2.40s;"></div><div class="sf sf3" style="top:-4.54em; left:53.74vw; animation-delay:-16.31s;"></div><div class="sf sf3" style="top:-4.82em; left:89.42vw; animation-delay:-17.39s;"></div><div class="sf sf1" style="top:-2.93em; left:40.05vw; animation-delay:-0.85s;"></div><div class="sf sf3" style="top:-4.94em; left:70.75vw; animation-delay:-7.94s;"></div><div class="sf sf0" style="top:-1.93em; left:44.77vw; animation-delay:-8.79s;"></div><div class="sf sf3" style="top:-4.65em; left:18.88vw; animation-delay:-11.51s;"></div><div class="sf sf1" style="top:-2.68em; left:6.18vw; animation-delay:-0.35s;"></div><div class="sf sf0" style="top:-1.80em; left:47.93vw; animation-delay:-16.81s;"></div><div class="sf sf1" style="top:-2.68em; left:44.60vw; animation-delay:-16.41s;"></div><div class="sf sf1" style="top:-2.16em; left:38.65vw; animation-delay:-7.02s;"></div><div class="sf sf2" style="top:-3.08em; left:33.24vw; animation-delay:-6.09s;"></div><div class="sf sf1" style="top:-2.80em; left:85.34vw; animation-delay:-11.04s;"></div><div class="sf sf0" style="top:-1.72em; left:40.19vw; animation-delay:-14.57s;"></div><div class="sf sf2" style="top:-3.35em; left:6.85vw; animation-delay:-1.87s;"></div><div class="sf sf0" style="top:-1.79em; left:48.73vw; animation-delay:-5.57s;"></div><div class="sf sf0" style="top:-1.96em; left:49.53vw; animation-delay:-14.93s;"></div><div class="sf sf2" style="top:-3.31em; left:58.25vw; animation-delay:-14.49s;"></div><div class="sf sf2" style="top:-3.58em; left:72.09vw; animation-delay:-12.56s;"></div><div class="sf sf1" style="top:-2.24em; left:55.81vw; animation-delay:-19.29s;"></div><div class="sf sf0" style="top:-1.83em; left:34.32vw; animation-delay:-15.84s;"></div><div class="sf sf2" style="top:-3.95em; left:78.40vw; animation-delay:-19.15s;"></div><div class="sf sf0" style="top:-1.38em; left:88.43vw; animation-delay:-19.91s;"></div><div class="sf sf0" style="top:-1.90em; left:48.44vw; animation-delay:-3.49s;"></div><div class="sf sf3" style="top:-4.35em; left:48.47vw; animation-delay:-4.80s;"></div><div class="sf sf3" style="top:-4.99em; left:14.52vw; animation-delay:-17.95s;"></div><div class="sf sf0" style="top:-1.04em; left:74.88vw; animation-delay:-10.19s;"></div><div class="sf sf3" style="top:-4.48em; left:66.83vw; animation-delay:-7.74s;"></div><div class="sf sf0" style="top:-1.12em; left:27.40vw; animation-delay:-19.53s;"></div><div class="sf sf2" style="top:-3.72em; left:80.77vw; animation-delay:-3.99s;"></div><div class="sf sf1" style="top:-2.94em; left:70.90vw; animation-delay:-16.84s;"></div><div class="sf sf2" style="top:-3.02em; left:9.15vw; animation-delay:-14.02s;"></div><div class="sf sf1" style="top:-2.91em; left:4.80vw; animation-delay:-17.38s;"></div><div class="sf sf3" style="top:-4.58em; left:77.64vw; animation-delay:-2.65s;"></div><div class="sf sf3" style="top:-4.99em; left:42.53vw; animation-delay:-12.84s;"></div><div class="sf sf1" style="top:-2.47em; left:74.15vw; animation-delay:-8.33s;"></div><div class="sf sf2" style="top:-3.21em; left:18.16vw; animation-delay:-10.80s;"></div><div class="sf sf0" style="top:-1.48em; left:52.08vw; animation-delay:-5.40s;"></div><div class="sf sf2" style="top:-3.88em; left:43.22vw; animation-delay:-8.66s;"></div><div class="sf sf0" style="top:-1.21em; left:73.19vw; animation-delay:-1.41s;"></div><div class="sf sf3" style="top:-4.37em; left:11.64vw; animation-delay:-2.92s;"></div><div class="sf sf2" style="top:-3.75em; left:59.42vw; animation-delay:-7.68s;"></div><div class="sf sf3" style="top:-4.89em; left:1.64vw; animation-delay:-16.12s;"></div><div class="sf sf1" style="top:-2.66em; left:53.78vw; animation-delay:-19.69s;"></div><div class="sf sf0" style="top:-1.33em; left:22.90vw; animation-delay:-5.86s;"></div><div class="sf sf2" style="top:-3.99em; left:8.69vw; animation-delay:-14.60s;"></div><div class="sf sf1" style="top:-2.34em; left:52.74vw; animation-delay:-2.99s;"></div><div class="sf sf2" style="top:-3.87em; left:45.47vw; animation-delay:-12.97s;"></div><div class="sf sf0" style="top:-1.19em; left:14.65vw; animation-delay:-9.22s;"></div><div class="sf sf1" style="top:-2.74em; left:26.66vw; animation-delay:-3.36s;"></div><div class="sf sf3" style="top:-4.18em; left:72.09vw; animation-delay:-8.58s;"></div><div class="sf sf3" style="top:-4.79em; left:88.00vw; animation-delay:-10.00s;"></div><div class="sf sf3" style="top:-4.09em; left:65.20vw; animation-delay:-11.48s;"></div><div class="sf sf0" style="top:-1.99em; left:89.54vw; animation-delay:-16.12s;"></div><div class="sf sf1" style="top:-2.06em; left:42.14vw; animation-delay:-15.35s;"></div><div class="sf sf3" style="top:-4.59em; left:14.86vw; animation-delay:-18.48s;"></div><div class="sf sf0" style="top:-1.12em; left:41.09vw; animation-delay:-11.83s;"></div><div class="sf sf2" style="top:-3.85em; left:70.24vw; animation-delay:-2.37s;"></div><div class="sf sf1" style="top:-2.02em; left:53.00vw; animation-delay:-16.53s;"></div><div class="sf sf1" style="top:-2.26em; left:9.53vw; animation-delay:-10.54s;"></div><div class="sf sf3" style="top:-4.16em; left:83.79vw; animation-delay:-4.92s;"></div><div class="sf sf0" style="top:-1.74em; left:42.70vw; animation-delay:-14.26s;"></div>
</main>
